## Date : 1st July 2022
## Author : CJ
## Task : Create a web crawler that gets the reviews of top ten competetors from playstore

Okay so lets start this. Im waiting for Siddharth to send me the document so till I get that, Im gonna try spamming google playstore site with requests to try and see if they block me. Lets just hope I dont get flagged lol.

In [2]:
import requests
from requests.exceptions import ConnectionError
from bs4 import BeautifulSoup as bs
import urllib.parse
import pandas as pd
import numpy as np
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [19]:
# Lets just pause it for right now, I can add the companies manually
# df = pd.read_csv('Datasets/Competitor Intelligence.csv')
# listOfCompanies = df.columns.to_list()[3:]
# print(listOfCompanies)
listOfCompanies = ['INDwealth', 'Paytm Money', 'Groww', 'Wizely', 'Goalwise ', 'Scripbox', 'Orowealth', 'Cube Wealth', 'Fisdom', 'FundsIndia', 'Wealthy', 'Sqrrl', 'Arthayantra', 'Fintoo', 'ET Money', 'Dezerve', 'Kuvera']

In [22]:
def findIds(listOfCompanies, debug=False):
    if debug:
        print("Trying to find Ids for apps:")
    baseUrl = 'https://www.google.com/search?q='
    ids = {}
    for company in listOfCompanies:
        url = baseUrl+company+'playstore'+'reviews'+'finance'
        ids[company] = []
        try:
            response = requests.get(url)
            content = bs(response.text, "lxml")
            div_content = content.find_all('a')
            if debug:
                print("Hit : "+company)
            for ele in div_content:
                for word in str(ele).split():
                    if "https://play.google.com/" in word:
                        ansUrl = urllib.parse.unquote_plus(ele['href'])
                        initial = ansUrl.find('id=')
                        final = ansUrl[initial:].find('&')
                        if debug:
                            print("appending")
                        ids[company].append(ansUrl[initial+3: initial+final])
        except ConnectionError:
            if debug:
                print("Miss : "+company)
                print("URL trid : "+url)
            
    return ids

Okay, finally made it to work. So basically this function takes a list of companies and then does a google search for them. I take the first 10 links and see if there are any links going to the playstore website from there. Then I just take the app id from those links and add them to the dictionary.

Now for the next task, Im going to try and hit all those playstore pages using those ids.

Amazing, all hits, ill now create a parser and try to get reviews.

EOD-----------------------------------------------------------------------------------------------------------------

Date : 2nd July 2022

Okay so a recap, my laptop crashed and some shit just got lost, but it was just testing data. So I have to type this log again. I tried getting reviews from requests, but those guys encrypted it, so that was a dead end (unless I spend time decrypting it <i> Note to self, try maybe writing a decryption algo later</i>)

Anyway, Ashutosh suggested that I should look into selenium so I did that, and damn this thing is powerful. I should try automating tinder with this later. But back to the topic, I read a few posts about it, and I think this can work. Also why cant the traffic in Bangalore stay constant dude, I left early cos I thought there would be traffic but apparantly I arrived too early, but gladly Kumar was here, and hes an amazing person, so I was not bored.

Okay update time, I could have worked from home, cos apparantly not many people come on saturdays ffs. But anyway, the selenium thing is working fine, and now its time to integrate it with the links part of the code. But now that I think about it, dude i could have gotten links in just a few lines by using selenium lol, but anyway, my way is muchhhhhh more faster, so cheers.

In [23]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup as bs

In [24]:
#Only needed cos I was dumb enough to use kali for everyday-use
drivLoc = '/usr/bin/chromedriver'
binLoc = '/usr/bin/google-chrome'

In [25]:
options = webdriver.ChromeOptions()
options.binary_location = binLoc
#To run without browser
# options.add_argument('headless')

In [42]:
def getReviewsObj(idDict, depth, debug=False):
    if debug:
        print("Trying to fetch reviews:")
    reviewDict = {}
    driver = webdriver.Chrome(executable_path=drivLoc, options=options)
    for company, ids in idDict.items():
        for id in ids:
            if company not in reviewDict:
                try:
                    reviewDict[company]=[]
                    url = 'https://play.google.com/store/apps/details?id='+id
                    driver.get(url)
                    arrow = driver.find_element_by_xpath('/html/body/c-wiz[2]/div/div/div[1]/div[2]/div/div[1]/c-wiz[4]/section/header/div/div[2]/button/i')
                    arrow.click()
                    reviews = driver.find_element_by_css_selector('.VfPpkd-cnG4Wd > div:nth-child(1)')
                    time.sleep(1)
                    crawlWriter = open('reviewData/'+company, 'a')
                    actions = ActionChains(driver)
                    actions.click(reviews)
                    prevLen = 0 
                    i=0
                    j=0
                    while i<depth:
                        if j > 20*depth:
                            break
                        actions.send_keys(Keys.PAGE_DOWN).perform()
                        j+=1
                        reviewsList = bs(reviews.get_attribute('innerHTML')).find_all('div', {'class':'RHo1pe'})#
                        if len(reviewsList) > prevLen:
                            time.sleep(0.5)
                            for review in reviewsList[prevLen:]:
                                crawlWriter.write("%s\n" % review)
                            prevLen = len(reviewsList)
                            i+=1
                        time.sleep(0.1)
                    if debug:
                        print("HIT : "+id)
                        print(reviews.text[:20])
                except:
                    if debug:
                        print("FUCKED : "+id)
    time.sleep(3)
    driver.quit()
                        
    return reviewDict

In [41]:
start = time.time()
reviewDict = {}
idDict = findIds(listOfCompanies)
getReviewsObj(idDict, 3)
end = time.time()
print("Exectution time: "+str(end-start))


/tmp/ipykernel_17746/2125726736.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=drivLoc, options=options)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/c-wiz[2]/div/div/div[1]/div[2]/div/div[1]/c-wiz[4]/section/header/div/div[2]/button/i"}
  (Session info: chrome=103.0.5060.53)


So, took about 54 seconds, just for 8 apps. But most of the blame goes to my shitty laptop. With something more powerful, its still not going to be that bad. Now the hard part, well, imm gonna try and add all those reviews I downloaded to a file. Letss see if my laptop will be able to cook food.

In [12]:
list1 = []
reader = open('reviewData/Paytm Money')
lines = reader.readlines()
for line in  lines:
    item = line[:-1]
    if len(bs(item).text.strip())>500:
        list1.append(item)
print(len(list1))

159


Okay time for another update, well, I cant go househunting untill ajitesh returns, so I started working on the parser, and I made the code much more efficient, since its just storing the reviews now, no other crap. And it took just 320 seconds to collect about 160 reviews for each of the provided companies.
Now Ill just try and put the parser in a function.

In [20]:
def parseReviewFiles(filename):
    reviewList = []
    try:
        reader = open('Test Data/'+filename)
        lines = reader.readlines()
    except:
        print("Couldnot open file")
        return []
    for line in  lines:
        reviewDict = {}
        item = line[:-1]
        bsObj = bs(item)
        if len(str(bsObj))>200:
            try:
                reviewDict['CustUname'] = bsObj.find_all('div', {'class':'X5PpBb'})[0].text
#                 reviewDict['Date'] = bsObj.find_all('div', {'span':'bp9Aid'})[0].text
                reviewDict['ReviewText'] = bsObj.find_all('div', {'class':'h3YV2d'})[0].text
                reviewDict['Rating'] = bsObj.find_all('div', {'class':'iXRFPc'})[0]
                reviewList.append(reviewDict)
            except:
                continue
    print(len(reviewList))
    return reviewList

In [22]:
rl = parseReviewFiles('Paytm Money')

412


In [25]:
for key, val in rl[0].items():
    print(key+" : "+str(val))

CustUname : Venkat Karthik
ReviewText : Excellent app, very user friendly and very easy to do any investment. One stop for all investments. But only challenge I face with the app is customer support. Not at all user friendly. So as long as there are no issues all will be good if issues are there then it's difficult to get resolutions for those.
Rating : <div aria-label="Rated 4 stars out of five stars" class="iXRFPc" role="img"><span aria-hidden="true" class="F7XJmb" data-number="1" jsname="fI6EEc"><span aria-hidden="true" class="Z1Dz7b"><svg viewbox="0 0 24 24" xmlns="http://www.w3.org/2000/svg"><path d="M0 0h24v24H0z" fill="none"></path><path d="M0 0h24v24H0z" fill="none"></path><path d="M12 17.27L18.18 21l-1.64-7.03L22 9.24l-7.19-.61L12 2 9.19 8.63 2 9.24l5.46 4.73L5.82 21z"></path></svg></span></span><span aria-hidden="true" class="F7XJmb" data-number="2" jsname="fI6EEc"><span aria-hidden="true" class="Z1Dz7b"><svg viewbox="0 0 24 24" xmlns="http://www.w3.org/2000/svg"><path d="M0 

In [46]:
# X5PpBb UNAME
# iXRFPc star (use lable)
# bp9Aid date(span)
# h3YV2d review
# ras4vb reply
# I9Jtec reply date